# Expresion
Datos de expresión extraidos de:
https://ccb-web.cs.uni-saarland.de/tissueatlas2/downloads

In [18]:
import pandas as pd 
import seaborn as sns
import multiprocessing as mp
from tqdm import tqdm
import ast
import matplotlib.pyplot as plt

In [19]:
dft = pd.read_csv('../../Data/Estratos/Pathways/Contribution_miRNA_time_pathwayGO_BP.tsv', sep = '\t')
dfv = pd.read_csv('../../Data/Estratos/Pathways/Contribution_miRNA_variant_pathwayGO_BP.tsv', sep = '\t')
dfs = pd.read_csv('../../Data/Estratos/Pathways/Contribution_miRNA_spectral_pathwayGO_BP.tsv', sep = '\t')
dfsp = pd.read_csv('../../Data/Estratos/Pathways/Contribution_miRNA_spectrum_pathwayGO_BP.tsv', sep = '\t')
dfgt = pd.read_csv('../../Data/Estratos/Pathways/Contribution_miRNA_ConstantClusters_pathwayGO_BP.tsv', sep = '\t')

dfv.head()

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR
0,circadian behavior,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85"
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85"
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85"
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85"
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85"


In [20]:
# estan en el primero, pero no en el segundo 
#len(set(dft[dft['Variant']=='(2010-12-05,2020-03-01]']['miRNA']) - set(dft[dft['Variant']=='Global']['miRNA']))

In [21]:
df_expression = pd.read_csv('../../Data/hsa_snc_expression.csv')
df_expression.head()

,source,norm,body,organ,tissue,type,acc,expression,species
0,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-1-1,88.905542,hsa
1,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-10-1,0.000000,hsa
2,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-10-2,0.000000,hsa
3,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-11-1,47.789356,hsa
4,ngs,rpmm,3,adipocyte,adipocyte,trna,tRNA-Ala-AGC-12-1,0.000000,hsa


In [22]:
tissues_ACE2 = ['brain', 'esophagus', 'lung', 'heart', 'kidney', 'stomach', 'gallbladder', 'liver', 'pancreas', 'bowel', 'vein', 'artery']
df_expression_sars = df_expression[df_expression['organ'].isin(tissues_ACE2)]
df_expression_sars = df_expression_sars[df_expression_sars['expression'] > 0]
df_expression_sars = df_expression_sars[df_expression_sars['type'] == 'mirna']
df_expression_sars

,source,norm,body,organ,tissue,type,acc,expression,species
171936,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7a-2-3p,0.500669,hsa
171937,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7a-3p,95.600018,hsa
171938,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7a-5p,2403.045671,hsa
171939,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7b-3p,13.907480,hsa
171940,ngs,rpmm,1,kidney,kidney,mirna,hsa-let-7b-5p,9122.861906,hsa
...,...,...,...,...,...,...,...,...,...
16411643,ngs,rpmm,3,liver,liver,mirna,hsa-miR-9986,0.251370,hsa
16411644,ngs,rpmm,3,liver,liver,mirna,hsa-miR-99a-3p,117.724918,hsa
16411645,ngs,rpmm,3,liver,liver,mirna,hsa-miR-99a-5p,3356.039952,hsa
16411646,ngs,rpmm,3,liver,liver,mirna,hsa-miR-99b-3p,6.591478,hsa


In [23]:
def extract_expression(miRNA):
    if miRNA in list(df_expression_sars['acc']):
        keys =  df_expression_sars[df_expression_sars['acc'] == miRNA]['organ']
        values = df_expression_sars[df_expression_sars['acc'] == miRNA]['expression']
        dictionary = dict(zip(keys, values))
        return dictionary
    else:
        return 0

In [24]:
# Crea un pool de procesos
pool = mp.Pool(mp.cpu_count())
pbar = tqdm(total=len(dft['miRNA']))
def update(*a):
    pbar.update()
results = []
for miRNA in dft['miRNA']:
    result = pool.apply_async(extract_expression, args=(miRNA,), callback=update)
    results.append(result)
pool.close()
pool.join()
dft['Expression'] = [result.get() for result in results]
pbar.close()

dft.head()

100%|██████████| 137914/137914 [12:45<00:00, 180.19it/s]


,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression
0,circadian behavior,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."


In [25]:
# Crea un pool de procesos
pool = mp.Pool(mp.cpu_count())
pbar = tqdm(total=len(dfv['miRNA']))
def update(*a):
    pbar.update()
results = []
for miRNA in dfv['miRNA']:
    result = pool.apply_async(extract_expression, args=(miRNA,), callback=update)
    results.append(result)
pool.close()
pool.join()
dfv['Expression'] = [result.get() for result in results]
pbar.close()

dfv.head()

100%|██████████| 551430/551430 [50:26<00:00, 182.20it/s]


,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression
0,circadian behavior,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946..."


In [26]:
# Crea un pool de procesos
pool = mp.Pool(mp.cpu_count())
pbar = tqdm(total=len(dfs['miRNA']))
def update(*a):
    pbar.update()
results = []
for miRNA in dfs['miRNA']:
    result = pool.apply_async(extract_expression, args=(miRNA,), callback=update)
    results.append(result)
pool.close()
pool.join()
dfs['Expression'] = [result.get() for result in results]
pbar.close()

dfs.head()

100%|██████████| 97541/97541 [08:55<00:00, 182.30it/s]


,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression
0,circadian behavior,0.034054,hsa-miR-638,G2,-34.5,0.84,5'UTR,>G2:67-125,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,G2,-34.5,0.84,5'UTR,>G2:67-125,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,G2,-34.5,0.84,5'UTR,>G2:67-125,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,G2,-34.5,0.84,5'UTR,>G2:67-125,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,G2,-34.5,0.84,5'UTR,>G2:67-125,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."


In [27]:
# Crea un pool de procesos
pool = mp.Pool(mp.cpu_count())
pbar = tqdm(total=len(dfsp['miRNA']))
def update(*a):
    pbar.update()
results = []
for miRNA in dfsp['miRNA']:
    result = pool.apply_async(extract_expression, args=(miRNA,), callback=update)
    results.append(result)
pool.close()
pool.join()
dfsp['Expression'] = [result.get() for result in results]
pbar.close()

dfsp.head()

100%|██████████| 60948/60948 [05:29<00:00, 184.91it/s]


,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression
0,circadian behavior,0.034054,hsa-miR-638,Gs2,-34.5,0.84,5'UTR,>Gs2:45-173,"45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,Gs2,-34.5,0.84,5'UTR,>Gs2:45-173,"45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,Gs2,-34.5,0.84,5'UTR,>Gs2:45-173,"45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,Gs2,-34.5,0.84,5'UTR,>Gs2:45-173,"45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,Gs2,-34.5,0.84,5'UTR,>Gs2:45-173,"45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946..."


In [28]:
# Crea un pool de procesos
pool = mp.Pool(mp.cpu_count())
pbar = tqdm(total=len(dfgt['miRNA']))
def update(*a):
    pbar.update()
results = []
for miRNA in dfgt['miRNA']:
    result = pool.apply_async(extract_expression, args=(miRNA,), callback=update)
    results.append(result)
pool.close()
pool.join()
dfgt['Expression'] = [result.get() for result in results]
pbar.close()

dfgt.head()

100%|██████████| 37529/37529 [03:26<00:00, 181.99it/s]


,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression
0,circadian behavior,0.034054,hsa-miR-638,Gst1,-34.5,0.84,5'UTR,>Gst1:67-209,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,Gst1,-34.5,0.84,5'UTR,>Gst1:67-209,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,Gst1,-34.5,0.84,5'UTR,>Gst1:67-209,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,Gst1,-34.5,0.84,5'UTR,>Gst1:67-209,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,Gst1,-34.5,0.84,5'UTR,>Gst1:67-209,"23,47","{'bowel': 0.0834956488747458, 'brain': 0.06946..."


In [29]:
dft.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_time.tsv', index = False, sep = '\t')
dfv.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_variant.tsv', index = False, sep = '\t')
dfs.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectral.tsv', index = False, sep = '\t')
dfsp.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectrum.tsv', index = False, sep = '\t')
dfgt.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_ContantClusters.tsv', index = False, sep = '\t')

# Preprocessing

In [30]:
dft = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_time.tsv', sep = '\t')
dfv = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_variant.tsv', sep = '\t')
dfs = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectral.tsv', sep = '\t')
dfsp = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectrum.tsv', sep = '\t')
dfgt = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_ContantClusters.tsv', sep = '\t')

len(dfv)

551430

In [31]:
dft['Expression'] = dft['Expression'].apply(ast.literal_eval)
expanded_dft = dft['Expression'].apply(pd.Series)
dft = pd.concat([dft, expanded_dft], axis=1)


dfv['Expression'] = dfv['Expression'].apply(ast.literal_eval)
expanded_dfv = dfv['Expression'].apply(pd.Series)
dfv = pd.concat([dfv, expanded_dfv], axis=1)

dfs['Expression'] = dfs['Expression'].apply(ast.literal_eval)
expanded_dfs = dfs['Expression'].apply(pd.Series)
dfs = pd.concat([dfs, expanded_dfs], axis=1)

dfsp['Expression'] = dfsp['Expression'].apply(ast.literal_eval)
expanded_dfsp = dfsp['Expression'].apply(pd.Series)
dfsp = pd.concat([dfsp, expanded_dfsp], axis=1)

dfgt['Expression'] = dfgt['Expression'].apply(ast.literal_eval)
expanded_dfgt = dfgt['Expression'].apply(pd.Series)
dfgt = pd.concat([dfgt, expanded_dfgt], axis=1)

dfv.head()

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression,...,lung,esophagus,kidney,artery,heart,vein,pancreas,stomach,liver,0
0,circadian behavior,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,20A,-34.5,0.84,5'UTR,>20A:29-173,"61,85","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
dft.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_time.tsv', index = False, sep = '\t')
dfv.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_variant.tsv', index = False, sep = '\t')
dfs.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectral.tsv', index = False, sep = '\t')
dfsp.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectrum.tsv', index = False, sep = '\t')
dfgt.to_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_ContantClusters.tsv', index = False, sep = '\t')

# Filtro

In [33]:
dft = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_time.tsv', sep = '\t')
dfv = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_variant.tsv', sep = '\t')
dfs = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectral.tsv', sep = '\t')
dfsp = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_spectrum.tsv', sep = '\t')
dfgt = pd.read_csv('../../Data/Estratos/Pathways_Expression/miRNA_expression_pathways_ContantClusters.tsv', sep = '\t')
dft.head()

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression,...,lung,esophagus,kidney,artery,heart,vein,pancreas,stomach,liver,0
0,circadian behavior,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,glycerophospholipid biosynthetic process,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,negative regulation of G1/S transition of mito...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,negative regulation of cell cycle G1/S phase t...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,negative regulation of epithelial to mesenchym...,0.034054,hsa-miR-638,"(2022-11-01,2023-01-01]",-34.5,0.84,5'UTR,">(2022-11-01,2023-01-01]:45-209","45,69","{'bowel': 0.0834956488747458, 'brain': 0.06946...",...,0.029673,0.034877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
dfft = dft[dft['lung']>10]
dffv = dfv[dfv['lung']>10]
dffs = dfs[dfs['lung']>10]
dffsp = dfsp[dfsp['lung']>10]
dffgt = dfgt[dfgt['lung']>10]

dfft

,function,p_value,miRNA,Variant,DuplexEnergy,Jaccard,Region,UTR_region,InteractionRegionUTR,Expression,...,lung,esophagus,kidney,artery,heart,vein,pancreas,stomach,liver,0
1046,cellular response to parathyroid hormone stimulus,0.008178,hsa-miR-381-3p,"(2010-12-05,2020-03-01]",-20.5,0.812500,3'UTR,">(2010-12-05,2020-03-01]:29763-29844","13,28","{'kidney': 47.3691337301885, 'brain': 282.3801...",...,12.382684,26.206047,47.369134,6.166493,28.999434,17.336111,26.912480,128.365923,36.057620,NaN
1047,negative regulation of DNA-binding transcripti...,0.008178,hsa-miR-381-3p,"(2010-12-05,2020-03-01]",-20.5,0.812500,3'UTR,">(2010-12-05,2020-03-01]:29763-29844","13,28","{'kidney': 47.3691337301885, 'brain': 282.3801...",...,12.382684,26.206047,47.369134,6.166493,28.999434,17.336111,26.912480,128.365923,36.057620,NaN
1048,negative regulation of ossification,0.013669,hsa-miR-381-3p,"(2010-12-05,2020-03-01]",-20.5,0.812500,3'UTR,">(2010-12-05,2020-03-01]:29763-29844","13,28","{'kidney': 47.3691337301885, 'brain': 282.3801...",...,12.382684,26.206047,47.369134,6.166493,28.999434,17.336111,26.912480,128.365923,36.057620,NaN
1049,negative regulation of osteoblast differentiation,0.008178,hsa-miR-381-3p,"(2010-12-05,2020-03-01]",-20.5,0.812500,3'UTR,">(2010-12-05,2020-03-01]:29763-29844","13,28","{'kidney': 47.3691337301885, 'brain': 282.3801...",...,12.382684,26.206047,47.369134,6.166493,28.999434,17.336111,26.912480,128.365923,36.057620,NaN
1050,osteoblast differentiation,0.021794,hsa-miR-381-3p,"(2010-12-05,2020-03-01]",-20.5,0.812500,3'UTR,">(2010-12-05,2020-03-01]:29763-29844","13,28","{'kidney': 47.3691337301885, 'brain': 282.3801...",...,12.382684,26.206047,47.369134,6.166493,28.999434,17.336111,26.912480,128.365923,36.057620,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137864,signaling,0.040279,hsa-miR-32-5p,"(2023-01-01,2023-07-01]",-10.0,0.846154,3'UTR,">(2023-01-01,2023-07-01]:29824-29823","10,22","{'kidney': 986.1392385648332, 'brain': 499.595...",...,334.138224,185.935100,986.139239,161.922394,678.337589,395.051669,527.913189,1104.384081,772.599449,NaN
137865,system development,0.036244,hsa-miR-32-5p,"(2023-01-01,2023-07-01]",-10.0,0.846154,3'UTR,">(2023-01-01,2023-07-01]:29824-29823","10,22","{'kidney': 986.1392385648332, 'brain': 499.595...",...,334.138224,185.935100,986.139239,161.922394,678.337589,395.051669,527.913189,1104.384081,772.599449,NaN
137866,telencephalon development,0.019501,hsa-miR-32-5p,"(2023-01-01,2023-07-01]",-10.0,0.846154,3'UTR,">(2023-01-01,2023-07-01]:29824-29823","10,22","{'kidney': 986.1392385648332, 'brain': 499.595...",...,334.138224,185.935100,986.139239,161.922394,678.337589,395.051669,527.913189,1104.384081,772.599449,NaN
137867,tube formation,0.038178,hsa-miR-32-5p,"(2023-01-01,2023-07-01]",-10.0,0.846154,3'UTR,">(2023-01-01,2023-07-01]:29824-29823","10,22","{'kidney': 986.1392385648332, 'brain': 499.595...",...,334.138224,185.935100,986.139239,161.922394,678.337589,395.051669,527.913189,1104.384081,772.599449,NaN


In [35]:
dfft.to_csv('../../Data/Estratos/Pathways_ExpressionFilter/Expanded_miRNAPredicted_time_Filter.tsv', sep = '\t', index = False)
dffv.to_csv('../../Data/Estratos/Pathways_ExpressionFilter/Expanded_miRNAPredicted_SinEstructuras_variant_Filter.tsv', sep = '\t', index = False)
dffs.to_csv('../../Data/Estratos/Pathways_ExpressionFilter/Expanded_miRNAPredicted_spectral_Filter.tsv', sep = '\t', index = False)
dffsp.to_csv('../../Data/Estratos/Pathways_ExpressionFilter/Expanded_miRNAPredicted_spectrum_Filter.tsv', sep = '\t', index = False)
dffgt.to_csv('../../Data/Estratos/Pathways_ExpressionFilter/Expanded_miRNAPredicted_ConstantClusters_Filter.tsv', sep = '\t', index = False)